In [5]:
import gurobipy as gp
from gurobipy import Model, GRB, quicksum
import numpy as np
import pandas as pd

ImportError: cannot import name 'log' from 'gurobipy' (/Users/oscar/anaconda3/lib/python3.11/site-packages/gurobipy/__init__.py)

In [3]:
# Load the data
file_path = "/Users/oscar/Documents/GitHub/Risk_Model_Research/ncd_milp/sim_milp/"
file_name = "sim_50_3_2_7_1_1_data.csv"  # Replace with your file name
df = pd.read_csv(file_path + file_name) 

# Preprocessing the data
#df = df.iloc[0:10,]
y = df.iloc[:, 0].values
X = df.iloc[:, 1:].values

# Parameters
n, p = X.shape
M = 1000
SK_pool = np.linspace(-5 * p, 5 * p + 1,10 * p + 2,dtype=int)
PI = np.linspace(0, 1, 100)[1:-1]  # Exclude 0 and 1

In [4]:
# Initialize Gurobi model
m = Model("model")

# Assuming p, n, SK_pool, X, y, and M are defined elsewhere in your code

# Variables
beta = m.addVars(p, vtype=GRB.INTEGER, lb=-5, ub=5, name="beta")
s = m.addVars(n, vtype=GRB.INTEGER, lb=-10, ub=10, name="s")
z_ik = m.addVars(n, len(SK_pool), vtype=GRB.BINARY, name="z_ik")
p_ik = m.addVars(n, len(SK_pool), vtype=GRB.CONTINUOUS, lb=0.0001, ub=0.9999, name="p_ik")
p_k = m.addVars(len(SK_pool), vtype=GRB.CONTINUOUS, lb=0.0001, ub=0.9999, name="p_k")

# Constraints
# Score constraint
for i in range(n):
    m.addConstr(quicksum(beta[j] * X[i, j] for j in range(p)) == s[i], name=f"score_constraint_{i}")

# z_ik constraint
for i in range(n):
    m.addConstr(quicksum(z_ik[i, k] for k in range(len(SK_pool))) == 1, name=f"z_ik_constraint_{i}")

# s_z_ik constraints
for i in range(n):
    for k in range(len(SK_pool)):
        m.addConstr(s[i] - k - M * (1 - z_ik[i, k]) <= 0, name=f"s_z_ik_constraint_1_{i}_{k}")
        m.addConstr(s[i] - k + M * (1 - z_ik[i, k]) >= 0, name=f"s_z_ik_constraint_2_{i}_{k}")

# p_ik constraints
for i in range(n):
    for k in range(len(SK_pool)):
        m.addConstr(p_ik[i, k] - p_k[k] <= M * (1 - z_ik[i, k]), name=f"p_ik_constraint_1_{i}_{k}")
        m.addConstr(p_ik[i, k] - p_k[k] >= -M * (1 - z_ik[i, k]), name=f"p_ik_constraint_2_{i}_{k}")

# Objective function
obj = quicksum(-y[i] * log(p_ik[i, k]) - (1 - y[i]) * log(1 - p_ik[i, k])
               for i in range(n) for k in range(len(SK_pool)))
m.setObjective(obj, GRB.MINIMIZE)

# Optimize model
m.optimize()


Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-25


NameError: name 'log' is not defined

In [ ]:

# Display variable values (if needed)
for v in m.getVars():
    print(f"{v.varName} = {v.x}")

print(f"Obj: {m.objVal}")